In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [3]:
df_sourceData = pd.read_csv('./现病史20190114内蒙古100患者.csv')
print(df_sourceData.info())
df_sourceData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
SEQ_ID            1000 non-null int64
PATIENT_ID        1000 non-null int64
PATIENT_NO        1000 non-null object
INPATIENT_NO      1000 non-null int64
INDEX2            0 non-null float64
PARENTNODENAME    1000 non-null object
SOURCEDATA        1000 non-null object
NODENAME          1000 non-null object
NODEVALUE         1000 non-null object
HOSPITAL_CODE     1000 non-null object
dtypes: float64(1), int64(3), object(6)
memory usage: 78.2+ KB
None


,SEQ_ID,PATIENT_ID,PATIENT_NO,INPATIENT_NO,INDEX2,PARENTNODENAME,SOURCEDATA,NODENAME,NODEVALUE,HOSPITAL_CODE
0,12409215,617387,10036263,722318,NaN,住院病历\现病史,患者5年前开始反复出现咳嗽、咳痰，白痰，伴气短，有时喘息，多于受凉后出现，就诊于当地卫生院给...,经常咳嗽_稳定期,有,12150000460028258G
1,12409215,617387,10036263,722318,NaN,住院病历\现病史,患者5年前开始反复出现咳嗽、咳痰，白痰，伴气短，有时喘息，多于受凉后出现，就诊于当地卫生院给...,双下肢水肿_加重期,未提及,12150000460028258G
2,12409215,617387,10036263,722318,NaN,住院病历\现病史,患者5年前开始反复出现咳嗽、咳痰，白痰，伴气短，有时喘息，多于受凉后出现，就诊于当地卫生院给...,经常胸闷_稳定期,未提及,12150000460028258G
3,12409215,617387,10036263,722318,NaN,住院病历\现病史,患者5年前开始反复出现咳嗽、咳痰，白痰，伴气短，有时喘息，多于受凉后出现，就诊于当地卫生院给...,经常活动后气促_稳定期,未提及,12150000460028258G
4,12409215,617387,10036263,722318,NaN,住院病历\现病史,患者5年前开始反复出现咳嗽、咳痰，白痰，伴气短，有时喘息，多于受凉后出现，就诊于当地卫生院给...,经常咳痰_稳定期,有,12150000460028258G


In [8]:
df_sourceData_withFullFeature = df_sourceData.groupby('SOURCEDATA').filter(lambda x:len(x) == 16)
print(df_sourceData_withFullFeature.info())
print(df_sourceData_withFullFeature.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 0 to 990
Data columns (total 10 columns):
SEQ_ID            928 non-null int64
PATIENT_ID        928 non-null int64
PATIENT_NO        928 non-null object
INPATIENT_NO      928 non-null int64
INDEX2            0 non-null float64
PARENTNODENAME    928 non-null object
SOURCEDATA        928 non-null object
NODENAME          928 non-null object
NODEVALUE         928 non-null object
HOSPITAL_CODE     928 non-null object
dtypes: float64(1), int64(3), object(6)
memory usage: 79.8+ KB
None
     SEQ_ID  PATIENT_ID PATIENT_NO  INPATIENT_NO  INDEX2 PARENTNODENAME  \
0  12409215      617387   10036263        722318     NaN       住院病历\现病史   
1  12409215      617387   10036263        722318     NaN       住院病历\现病史   
2  12409215      617387   10036263        722318     NaN       住院病历\现病史   
3  12409215      617387   10036263        722318     NaN       住院病历\现病史   
4  12409215      617387   10036263        722318     NaN       住院病历\现病史   



In [18]:
df_groupByIPNO = df_sourceData_withFullFeature.groupby('SOURCEDATA')
print(df_groupByIPNO.groups)

{'患者10余年前无明显诱因出现间断活动后气短，伴咳嗽、咳痰，休息后可缓解，病情反复发作，活动耐力逐渐下降，曾于我院诊断“慢性阻塞性肺疾病”，给予抗感染、化痰等对症治疗，症状好转后出院。1月前无明显诱因咳嗽、咳痰，为黄色粘痰，量少，痰利，活动后喘息气短，遂来我科就诊，我科拟以“慢性阻塞性肺疾病急性加重期”收入院。患者自发病以来偶有反酸、烧心，无咯血及呕血，无心悸及胸前区疼痛，精神、饮食及睡眠一般，大小便正常，体重未见明显改变。\n': Int64Index([848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860,
            861, 862, 863],
           dtype='int64'), '患者10天前出现肛周疼痛，因既往有痔疮病史故给予痔疮栓治疗，症状不缓解。一周前就诊于外院，给予相应治疗（具体方法及用药不详）,此后患者出现畏寒、发热，体温未测,自觉乏力,肛周疼痛明显,无大便习惯及性状改变。未做特殊处理,上诉症状逐渐加重并肛周疼痛加重。今为求诊治而入院,门诊结合病史及查体，以“直肠肿物”收入院。病后患者精神可，饮食一般,大便次数减少，小便如常。\n': Int64Index([240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
            253, 254, 255],
           dtype='int64'), '患者10天前感冒后出现咳嗽、咳痰症状，咳黄痰，无异味，无痰中带血，较易咳出，气短，活动后为著，无夜间阵发性呼吸困难，无发热、胸痛等症状，自服抗感冒药物疗效差，就诊呼市第一医院，胸片示左肺下野斑片状密度增高阴影，考虑肺炎可能，今为了进一步诊治入我院，自发病以来，无恶心、呕吐，无头晕、头痛，无发热、胸痛等症状，精神、饮食、睡眠欠佳，大小便正常。\n': Int64Index([192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204,
            205, 206, 207],
           dtype='int64'), '患者10天前无明显诱因出现胸憋、胸痛，

In [27]:
list_NODENAME = df_sourceData_withFullFeature[['NODENAME']].drop_duplicates()
list_NODENAME = list(list_NODENAME['NODENAME'])
print(list_NODENAME)

['经常咳嗽_稳定期', '双下肢水肿_加重期', '经常胸闷_稳定期', '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期', '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', '夜间阵发性呼吸困难_加重期', '经常喘息_稳定期']


In [28]:
list_NODEVALUE = df_sourceData_withFullFeature[['NODEVALUE']].drop_duplicates()
list_NODEVALUE = list(list_NODEVALUE['NODEVALUE'])
temp = list_NODEVALUE[1]
list_NODEVALUE[1] = list_NODEVALUE[2]
list_NODEVALUE[2] = temp
print(list_NODEVALUE)

['有', '无', '未提及']


In [34]:
def getLabelByGroup(group):
    label = [0]*(len(list_NODENAME)*len(list_NODEVALUE))
    for index, row in group.iterrows():
        name = row['NODENAME']
        nameIndex = list_NODENAME.index(name)
        value = row['NODEVALUE']
        valueIndex = list_NODEVALUE.index(value)
        labelIndex = len(list_NODEVALUE)*nameIndex+valueIndex
        label[labelIndex] = 1
        
    return label

In [38]:
list_NodeText = []
matrix_label = []
for name,group in df_groupByIPNO:
    list_NodeText.append(name)
    label = getLabelByGroup(group)
    matrix_label.append(label)
    

print(list_NodeText[:3])
print(matrix_label[:3])

['患者10余年前无明显诱因出现间断活动后气短，伴咳嗽、咳痰，休息后可缓解，病情反复发作，活动耐力逐渐下降，曾于我院诊断“慢性阻塞性肺疾病”，给予抗感染、化痰等对症治疗，症状好转后出院。1月前无明显诱因咳嗽、咳痰，为黄色粘痰，量少，痰利，活动后喘息气短，遂来我科就诊，我科拟以“慢性阻塞性肺疾病急性加重期”收入院。患者自发病以来偶有反酸、烧心，无咯血及呕血，无心悸及胸前区疼痛，精神、饮食及睡眠一般，大小便正常，体重未见明显改变。\n', '患者10天前出现肛周疼痛，因既往有痔疮病史故给予痔疮栓治疗，症状不缓解。一周前就诊于外院，给予相应治疗（具体方法及用药不详）,此后患者出现畏寒、发热，体温未测,自觉乏力,肛周疼痛明显,无大便习惯及性状改变。未做特殊处理,上诉症状逐渐加重并肛周疼痛加重。今为求诊治而入院,门诊结合病史及查体，以“直肠肿物”收入院。病后患者精神可，饮食一般,大便次数减少，小便如常。\n', '患者10天前感冒后出现咳嗽、咳痰症状，咳黄痰，无异味，无痰中带血，较易咳出，气短，活动后为著，无夜间阵发性呼吸困难，无发热、胸痛等症状，自服抗感冒药物疗效差，就诊呼市第一医院，胸片示左肺下野斑片状密度增高阴影，考虑肺炎可能，今为了进一步诊治入我院，自发病以来，无恶心、呕吐，无头晕、头痛，无发热、胸痛等症状，精神、饮食、睡眠欠佳，大小便正常。\n']
[[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [40]:
from bert_serving.client import BertClient
bc = BertClient(ip='localhost') 

In [41]:
# list_vecOfNodeText = bc.encode(list_NodeText)

# print(len(list_vecOfNodeText))
# print(len(list_vecOfNodeText[0]))
# print(list_vecOfNodeText[:5])

# import pickle
# file = open('list_vecOfNodeText.pkl','wb')
# pickle.dump(list_vecOfNodeText, file)
# file.close()

58
768
[[ 0.30084407  0.02137653 -0.1292413  ...  0.54416823 -0.52773553
   0.34503517]
 [ 0.50579524  0.03075447  0.11812722 ...  0.3939124  -0.10380789
   0.10666496]
 [ 0.6340203  -0.23786458 -0.13185601 ...  0.53493387 -0.27834788
   0.45843616]
 [ 0.6801551  -0.14320746  0.21322745 ...  0.6811169  -0.74772143
   0.40368515]
 [ 0.6146389  -0.17314756 -0.14192452 ...  0.40920568 -0.22043672
   0.18139254]]


In [42]:
import pickle

with open('list_vecOfNodeText.pkl','rb') as file:
    list_vecOfNodeText = pickle.load(file)
    
print(len(list_vecOfNodeText))
print(len(list_vecOfNodeText[0]))
print(list_vecOfNodeText[:5])

58
768
[[ 0.30084407  0.02137653 -0.1292413  ...  0.54416823 -0.52773553
   0.34503517]
 [ 0.50579524  0.03075447  0.11812722 ...  0.3939124  -0.10380789
   0.10666496]
 [ 0.6340203  -0.23786458 -0.13185601 ...  0.53493387 -0.27834788
   0.45843616]
 [ 0.6801551  -0.14320746  0.21322745 ...  0.6811169  -0.74772143
   0.40368515]
 [ 0.6146389  -0.17314756 -0.14192452 ...  0.40920568 -0.22043672
   0.18139254]]


In [58]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=len(list_vecOfNodeText[0])))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=len(list_NODENAME)*len(list_NODEVALUE), activation='relu'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 256)               196864    
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_23 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_24 (Dense)             (None, 48)                3120      
Total params: 241,136
Trainable params: 241,136
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
model.fit(np.array(list_vecOfNodeText), np.array(matrix_label), epochs=5000, batch_size=32, validation_split=0.1)

Train on 52 samples, validate on 6 samples
Epoch 1/5000
52/52 [==============================] - 0s 299us/step - loss: 96.3319 - acc: 0.0000e+00 - val_loss: 98.3012 - val_acc: 0.0000e+00
Epoch 2/5000
52/52 [==============================] - 0s 329us/step - loss: 96.3322 - acc: 0.0000e+00 - val_loss: 98.3000 - val_acc: 0.0000e+00
Epoch 3/5000
52/52 [==============================] - 0s 358us/step - loss: 96.3319 - acc: 0.0000e+00 - val_loss: 98.2985 - val_acc: 0.0000e+00
Epoch 4/5000
52/52 [==============================] - 0s 262us/step - loss: 96.3316 - acc: 0.0000e+00 - val_loss: 98.2968 - val_acc: 0.0000e+00
Epoch 5/5000
52/52 [==============================] - 0s 266us/step - loss: 96.3317 - acc: 0.0000e+00 - val_loss: 98.2951 - val_acc: 0.0000e+00
Epoch 6/5000
52/52 [==============================] - 0s 281us/step - loss: 96.3315 - acc: 0.0000e+00 - val_loss: 98.2938 - val_acc: 0.0000e+00
Epoch 7/5000
52/52 [==============================] - 0s 280us/step - loss: 96.3322 - acc: 0.

52/52 [==============================] - 0s 348us/step - loss: 96.3318 - acc: 0.0000e+00 - val_loss: 98.3072 - val_acc: 0.0000e+00
Epoch 58/5000
52/52 [==============================] - 0s 311us/step - loss: 96.3318 - acc: 0.0000e+00 - val_loss: 98.3082 - val_acc: 0.0000e+00
Epoch 59/5000
52/52 [==============================] - 0s 278us/step - loss: 96.3320 - acc: 0.0000e+00 - val_loss: 98.3086 - val_acc: 0.0000e+00
Epoch 60/5000
52/52 [==============================] - 0s 263us/step - loss: 96.3314 - acc: 0.0000e+00 - val_loss: 98.3092 - val_acc: 0.0000e+00
Epoch 61/5000
52/52 [==============================] - 0s 265us/step - loss: 96.3318 - acc: 0.0000e+00 - val_loss: 98.3102 - val_acc: 0.0000e+00
Epoch 62/5000
52/52 [==============================] - 0s 318us/step - loss: 96.3318 - acc: 0.0000e+00 - val_loss: 98.3111 - val_acc: 0.0000e+00
Epoch 63/5000
52/52 [==============================] - 0s 302us/step - loss: 96.3319 - acc: 0.0000e+00 - val_loss: 98.3111 - val_acc: 0.0000e+00

52/52 [==============================] - 0s 218us/step - loss: 96.3325 - acc: 0.0000e+00 - val_loss: 98.3220 - val_acc: 0.0000e+00
Epoch 114/5000
52/52 [==============================] - 0s 330us/step - loss: 96.3313 - acc: 0.0000e+00 - val_loss: 98.3208 - val_acc: 0.0000e+00
Epoch 115/5000
52/52 [==============================] - 0s 355us/step - loss: 96.3316 - acc: 0.0000e+00 - val_loss: 98.3197 - val_acc: 0.0000e+00
Epoch 116/5000
52/52 [==============================] - 0s 314us/step - loss: 96.3320 - acc: 0.0000e+00 - val_loss: 98.3189 - val_acc: 0.0000e+00
Epoch 117/5000
52/52 [==============================] - 0s 259us/step - loss: 96.3315 - acc: 0.0000e+00 - val_loss: 98.3191 - val_acc: 0.0000e+00
Epoch 118/5000
52/52 [==============================] - 0s 322us/step - loss: 96.3318 - acc: 0.0000e+00 - val_loss: 98.3197 - val_acc: 0.0000e+00
Epoch 119/5000
52/52 [==============================] - 0s 318us/step - loss: 96.3323 - acc: 0.0000e+00 - val_loss: 98.3201 - val_acc: 0.00

52/52 [==============================] - 0s 277us/step - loss: 96.3314 - acc: 0.0000e+00 - val_loss: 98.3145 - val_acc: 0.0000e+00
Epoch 170/5000
52/52 [==============================] - 0s 297us/step - loss: 96.3322 - acc: 0.0000e+00 - val_loss: 98.3138 - val_acc: 0.0000e+00
Epoch 171/5000
52/52 [==============================] - 0s 331us/step - loss: 96.3322 - acc: 0.0000e+00 - val_loss: 98.3128 - val_acc: 0.0000e+00
Epoch 172/5000
52/52 [==============================] - 0s 232us/step - loss: 96.3328 - acc: 0.0000e+00 - val_loss: 98.3116 - val_acc: 0.0000e+00
Epoch 173/5000
52/52 [==============================] - 0s 298us/step - loss: 96.3322 - acc: 0.0000e+00 - val_loss: 98.3110 - val_acc: 0.0000e+00
Epoch 174/5000
52/52 [==============================] - 0s 297us/step - loss: 96.3316 - acc: 0.0000e+00 - val_loss: 98.3108 - val_acc: 0.0000e+00
Epoch 175/5000
52/52 [==============================] - 0s 325us/step - loss: 96.3325 - acc: 0.0000e+00 - val_loss: 98.3104 - val_acc: 0.00

52/52 [==============================] - 0s 281us/step - loss: 96.3319 - acc: 0.0000e+00 - val_loss: 98.3163 - val_acc: 0.0000e+00
Epoch 226/5000
52/52 [==============================] - 0s 232us/step - loss: 96.3314 - acc: 0.0000e+00 - val_loss: 98.3160 - val_acc: 0.0000e+00
Epoch 227/5000
52/52 [==============================] - 0s 270us/step - loss: 96.3310 - acc: 0.0000e+00 - val_loss: 98.3156 - val_acc: 0.0000e+00
Epoch 228/5000
52/52 [==============================] - 0s 216us/step - loss: 96.3313 - acc: 0.0000e+00 - val_loss: 98.3154 - val_acc: 0.0000e+00
Epoch 229/5000
52/52 [==============================] - 0s 304us/step - loss: 96.3318 - acc: 0.0000e+00 - val_loss: 98.3155 - val_acc: 0.0000e+00
Epoch 230/5000
52/52 [==============================] - 0s 305us/step - loss: 96.3314 - acc: 0.0000e+00 - val_loss: 98.3158 - val_acc: 0.0000e+00
Epoch 231/5000
52/52 [==============================] - 0s 303us/step - loss: 96.3315 - acc: 0.0000e+00 - val_loss: 98.3156 - val_acc: 0.00

...

Epoch 3641/5000
52/52 [==============================] - 0s 251us/step - loss: 96.3305 - acc: 0.0000e+00 - val_loss: 98.3080 - val_acc: 0.0000e+00
Epoch 3642/5000
52/52 [==============================] - 0s 291us/step - loss: 96.3308 - acc: 0.0000e+00 - val_loss: 98.3090 - val_acc: 0.0000e+00
Epoch 3643/5000
52/52 [==============================] - 0s 362us/step - loss: 96.3300 - acc: 0.0000e+00 - val_loss: 98.3109 - val_acc: 0.0000e+00
Epoch 3644/5000
52/52 [==============================] - 0s 314us/step - loss: 96.3307 - acc: 0.0000e+00 - val_loss: 98.3131 - val_acc: 0.0000e+00
Epoch 3645/5000
52/52 [==============================] - 0s 250us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 98.3153 - val_acc: 0.0000e+00
Epoch 3646/5000
52/52 [==============================] - 0s 294us/step - loss: 96.3303 - acc: 0.0000e+00 - val_loss: 98.3177 - val_acc: 0.0000e+00
Epoch 3647/5000
52/52 [==============================] - 0s 304us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 9

Epoch 3697/5000
52/52 [==============================] - 0s 263us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 98.2989 - val_acc: 0.0000e+00
Epoch 3698/5000
52/52 [==============================] - 0s 240us/step - loss: 96.3304 - acc: 0.0000e+00 - val_loss: 98.2978 - val_acc: 0.0000e+00
Epoch 3699/5000
52/52 [==============================] - 0s 335us/step - loss: 96.3306 - acc: 0.0000e+00 - val_loss: 98.2967 - val_acc: 0.0000e+00
Epoch 3700/5000
52/52 [==============================] - 0s 334us/step - loss: 96.3308 - acc: 0.0000e+00 - val_loss: 98.2964 - val_acc: 0.0000e+00
Epoch 3701/5000
52/52 [==============================] - 0s 246us/step - loss: 96.3308 - acc: 0.0000e+00 - val_loss: 98.2963 - val_acc: 0.0000e+00
Epoch 3702/5000
52/52 [==============================] - 0s 311us/step - loss: 96.3303 - acc: 0.0000e+00 - val_loss: 98.2973 - val_acc: 0.0000e+00
Epoch 3703/5000
52/52 [==============================] - 0s 325us/step - loss: 96.3299 - acc: 0.0000e+00 - val_loss: 9

Epoch 3753/5000
52/52 [==============================] - 0s 326us/step - loss: 96.3303 - acc: 0.0000e+00 - val_loss: 98.2987 - val_acc: 0.0000e+00
Epoch 3754/5000
52/52 [==============================] - 0s 291us/step - loss: 96.3304 - acc: 0.0000e+00 - val_loss: 98.2974 - val_acc: 0.0000e+00
Epoch 3755/5000
52/52 [==============================] - 0s 247us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 98.2957 - val_acc: 0.0000e+00
Epoch 3756/5000
52/52 [==============================] - 0s 272us/step - loss: 96.3300 - acc: 0.0000e+00 - val_loss: 98.2937 - val_acc: 0.0000e+00
Epoch 3757/5000
52/52 [==============================] - 0s 290us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 98.2922 - val_acc: 0.0000e+00
Epoch 3758/5000
52/52 [==============================] - 0s 383us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 98.2909 - val_acc: 0.0000e+00
Epoch 3759/5000
52/52 [==============================] - 0s 312us/step - loss: 96.3324 - acc: 0.0000e+00 - val_loss: 9

Epoch 3809/5000
52/52 [==============================] - 0s 309us/step - loss: 96.3301 - acc: 0.0000e+00 - val_loss: 98.3137 - val_acc: 0.0000e+00
Epoch 3810/5000
52/52 [==============================] - 0s 363us/step - loss: 96.3299 - acc: 0.0000e+00 - val_loss: 98.3128 - val_acc: 0.0000e+00
Epoch 3811/5000
52/52 [==============================] - 0s 246us/step - loss: 96.3302 - acc: 0.0000e+00 - val_loss: 98.3113 - val_acc: 0.0000e+00
Epoch 3812/5000
52/52 [==============================] - 0s 260us/step - loss: 96.3301 - acc: 0.0000e+00 - val_loss: 98.3094 - val_acc: 0.0000e+00
Epoch 3813/5000
52/52 [==============================] - 0s 351us/step - loss: 96.3299 - acc: 0.0000e+00 - val_loss: 98.3077 - val_acc: 0.0000e+00
Epoch 3814/5000
52/52 [==============================] - 0s 317us/step - loss: 96.3305 - acc: 0.0000e+00 - val_loss: 98.3061 - val_acc: 0.0000e+00
Epoch 3815/5000
52/52 [==============================] - 0s 350us/step - loss: 96.3306 - acc: 0.0000e+00 - val_loss: 9

KeyboardInterrupt: 